In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import sklearn.metrics

import random
random.seed(7)


def generator(X, Y, batch_size=32, train=True):
    while True:
        for offset in range(0, len(X), batch_size):
            X_batch = np.stack(X[offset:offset+batch_size], axis=0)
            Y_batch = np.stack(Y[offset:offset+batch_size], axis=0)

            Y_batch_ = np.empty((Y_batch.shape[0], 2,2,10))
            for m in range(Y_batch.shape[0]):
                for i in range(10):
                    Y_batch_[m, :,:,i] = [[np.sum(Y_batch[m, :5, :5, i]), np.sum(Y_batch[m, :5, 5:, i])], [np.sum(Y_batch[m, 5:, :5, i]), np.sum(Y_batch[m, 5:, 5:, i])]]
            Y_batch_[Y_batch_ > 0] = 1

            if train:
                Y_f = np.array([Y_batch_[e].flatten() for e in range(Y_batch_.shape[0])])
                yield (X_batch, Y_f)
            else:
                yield X_batch

structure_ids = []
for line in open('./structures lists/structures human.txt', 'r'):
    line = line.strip('\n')
    structure_ids.append(line)
# for line in open('./structures lists/structures ecoli.txt', 'r'):
#     line = line.strip('\n')
#     structure_ids.append(line)
structure_ids.remove('4pkd')
structure_ids.remove('1a9n')
structure_ids.remove('2adc')
random.shuffle(structure_ids)
print(len(structure_ids))

X_train = []
X_test = []
Y_train = []
Y_test = []
num_aa_train = 0
num_aa_test = 0
num_train = int(len(structure_ids)*0.7)
for i, structure_id in enumerate(structure_ids):
    protein = np.load('../data/voxelized data 10x10x10/' + structure_id + '_protein.npy', mmap_mode='r')
    rna = np.load('../data/voxelized data 10x10x10/' + structure_id + '_rna_3D.npy', mmap_mode='r')
    na = 0
    pos = 0
    while (np.sum(rna[na]) > 0) and (na < len(rna)-1):
        pos +=1
        na +=1
    

    if i <= num_train:
        if pos > len(rna)/2:
            X_train.extend(protein[:, :, :, :, :3])
            Y_train.extend(rna)
            num_aa_train +=len(rna)
        else:
            X_train.extend(protein[:pos, :, :, :, :3])
            X_train.extend(protein[-pos:, :, :, :, :3])
            Y_train.extend(rna[:pos])
            Y_train.extend(rna[-pos:])
            num_aa_train +=2*pos
    else:
        if pos > len(rna)/2:
            X_test.extend(protein[:, :, :, :, :3])
            Y_test.extend(rna)
            num_aa_test +=len(rna)
        else:
            X_test.extend(protein[:pos, :, :, :, :3])
            X_test.extend(protein[-pos:, :, :, :, :3])
            Y_test.extend(rna[:pos])
            Y_test.extend(rna[-pos:])
            num_aa_test +=2*pos

Y_test = np.stack(Y_test, axis=0)
Y_test_ = np.empty((len(Y_test), 2,2,10))
for m in range(len(Y_test)):
    for i in range(10):
        Y_test_[m, :,:,i] = [[np.sum(Y_test[m, :5, :5, i]), np.sum(Y_test[m, :5, 5:, i])], [np.sum(Y_test[m, 5:, :5, i]), np.sum(Y_test[m, 5:, 5:, i])]]
Y_test_[Y_test_ > 0] = 1
   
n_steps_train = int(num_aa_train/400) 
n_steps_test = int(num_aa_test/400)

print(num_aa_train, num_aa_test)

generator_train = generator(X_train, Y_train, 400, True)
generator_test = generator(X_test, Y_test, 400, False)

ins = tf.keras.layers.Input((10, 10, 10, 3))
con1 = tf.keras.layers.Conv3D(filters=64, kernel_size=(3, 3, 3), padding='same', activation='relu')(ins)
con2 = tf.keras.layers.Conv3D(filters=32, kernel_size=(3, 3, 3), padding='same', activation='relu')(con1)
con3 = tf.keras.layers.Conv3D(filters=32, kernel_size=(3, 3, 3), padding='same', activation='relu')(con2)
maxp1 = tf.keras.layers.MaxPool3D(pool_size=(2, 2, 2))(con3)
con4 = tf.keras.layers.Conv3D(filters=32, kernel_size=(3, 3, 3), padding='same', activation='relu')(maxp1)
con5 = tf.keras.layers.Conv3D(filters=16, kernel_size=(3, 3, 3), padding='same', activation='relu')(con4)
con6 = tf.keras.layers.Conv3D(filters=16, kernel_size=(3, 3, 3), padding='same', activation='relu')(con5)
maxp2 = tf.keras.layers.MaxPool3D(pool_size=(2, 2, 2))(con6)
con7 = tf.keras.layers.Conv3D(filters=16, kernel_size=(3, 3, 3), padding='same', activation='relu')(maxp2)
con8 = tf.keras.layers.Conv3D(filters=8, kernel_size=(3, 3, 3), padding='same', activation='relu')(con7)
con9 = tf.keras.layers.Conv3D(filters=4, kernel_size=(3, 3, 3), padding='same', activation='relu')(con8)
maxp3 = tf.keras.layers.MaxPool3D(pool_size=(2, 2, 2))(con9)
batch = tf.keras.layers.BatchNormalization()(maxp3)
flat = tf.keras.layers.Flatten()(batch)
dens2 = tf.keras.layers.Dense(units=256, activation='relu')(flat)
drop2 = tf.keras.layers.Dropout(0.6)(dens2)
outs = tf.keras.layers.Dense(units=40, activation='sigmoid')(drop2)
model = tf.keras.models.Model(inputs=ins, outputs=outs)
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=0.00001), metrics=['accuracy'])

model.summary()

# checkpoint
# filepath="weights_best.hdf5"
# checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_acc', verbose=0, save_best_only=True, mode='max')
# callbacks_list = [checkpoint]

# model.fit(X_train, Y_train_f, validation_split=0.33, epochs=1, batch_size=200, callbacks=callbacks_list, verbose=0)
model.fit_generator(generator_train, steps_per_epoch=n_steps_train, epochs=500, callbacks=None, verbose=1, max_queue_size=2)

# model_best = model
# model_best.load_weights("weights_best.hdf5")
# print(model.evaluate(X_test, Y_test, verbose=0, batch_size=100))
# model_best.save('model_cnn_15_2.h5')
# Y_pred = model_best.predict(X_test, batch_size=200)
Y_pred = model.predict_generator(generator_test, steps=n_steps_test)
print(Y_pred.shape)
Y_pred_ = np.array([Y_pred[i].reshape((2,2,10)) for i in range(Y_pred.shape[0])])

#CNN
Y_pred_[Y_pred_ >= 0.5] = 1
Y_pred_[Y_pred_ < 0.5] = 0

print(Y_pred_.shape)
print(Y_test_.shape)
Y_test_ = Y_test_[:Y_pred_.shape[0]]

print('CNN: \n')
for i in range(10):
    confusion_matrix = [sklearn.metrics.confusion_matrix(Y_test_[:,l , c, i], Y_pred_[:,l , c, i]) for l in range(2) for c in range(2)]
    accuracy = [np.sum(np.trace(cm))/np.sum(cm) for cm in confusion_matrix]
    auc = [sklearn.metrics.roc_auc_score(Y_test_[:,l , c, i], Y_pred_[:,l , c, i]) for l in range(2) for c in range(2)]

    print(f'level {i}')
    for q in range(len(confusion_matrix)):
        print(confusion_matrix[q], np.round(accuracy[q], 2), np.round(auc[q], 2))

# baseline model
# predict all zeros; at least 50% correct predictions because there are 1/2 of negative examples
Y_pred_base = np.zeros(Y_test_.shape)

# po = np.sum(Y_train, axis=0)/Y_train.shape[0]
# po[po >= 0.5] = 1
# po[po < 0.5] = 0
# Y_pred_base = np.tile(po, (Y_test.shape[0],1))

Y_pred_base[Y_pred_base >= 0.5] = 1
Y_pred_base[Y_pred_base < 0.5] = 0
print(Y_pred_base.shape)
print(f'\n BASELINE MODEL: \n')
for i in range(10):
    confusion_matrix_base = [sklearn.metrics.confusion_matrix(Y_test_[:,l, c, i], Y_pred_base[:,l, c, i]) for l in range(2) for c in range(2)]
    accuracy_base = [np.sum(np.trace(cm))/np.sum(cm) for cm in confusion_matrix_base]
    auc_base = [sklearn.metrics.roc_auc_score(Y_test_[:,l, c, i], Y_pred_base[:,l, c, i]) for l in range(2) for c in range(2)]
    
    print(f'level {i}')
    for q in range(len(confusion_matrix_base)):
        print(confusion_matrix_base[q], np.round(accuracy_base[q], 2), np.round(auc_base[q], 2))

#random model
Y_pred_random = np.random.random(Y_test_.shape)
Y_pred_random[Y_pred_random >= 0.5] = 1
Y_pred_random[Y_pred_random < 0.5] = 0

print(f'\n RANDOM MODEL: \n')
for i in range(10):
    confusion_matrix_random = [sklearn.metrics.confusion_matrix(Y_test_[:,l, c, i], Y_pred_random[:,l, c, i]) for l in range(2) for c in range(2)]
    accuracy_random = [np.sum(np.trace(cm))/np.sum(cm) for cm in confusion_matrix_random]
    auc_random = [sklearn.metrics.roc_auc_score(Y_test_[:,l, c, i], Y_pred_random[:,l, c, i]) for l in range(2) for c in range(2)]

    print(f'level {i}')
    for q in range(len(confusion_matrix_random)):
        print(confusion_matrix_random[q], np.round(accuracy_random[q], 2), np.round(auc_random[q], 2))

230
83862 44057
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 10, 10, 10, 3)     0         
_________________________________________________________________
conv3d (Conv3D)              (None, 10, 10, 10, 64)    5248      
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 10, 10, 10, 32)    55328     
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 10, 10, 10, 32)    27680     
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 5, 5, 5, 32)       0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 5, 5, 5, 32)       27680     
_________________________________________________________________
conv3d_4 (Conv3D)            (None, 5, 5, 5, 16)       13840

209/209 [==============================] - 26s 122ms/step - loss: 0.4102 - acc: 0.8543
Epoch 56/500
209/209 [==============================] - 26s 124ms/step - loss: 0.4098 - acc: 0.8544
Epoch 57/500
209/209 [==============================] - 25s 119ms/step - loss: 0.4096 - acc: 0.8542
Epoch 58/500
209/209 [==============================] - 26s 123ms/step - loss: 0.4088 - acc: 0.8544
Epoch 59/500
209/209 [==============================] - 26s 124ms/step - loss: 0.4083 - acc: 0.8545
Epoch 60/500
209/209 [==============================] - 25s 120ms/step - loss: 0.4091 - acc: 0.8540
Epoch 61/500
209/209 [==============================] - 24s 115ms/step - loss: 0.4087 - acc: 0.8541
Epoch 62/500
209/209 [==============================] - 26s 124ms/step - loss: 0.4076 - acc: 0.8546
Epoch 63/500
209/209 [==============================] - 25s 120ms/step - loss: 0.4055 - acc: 0.8554
Epoch 64/500
209/209 [==============================] - 25s 119ms/step - loss: 0.4065 - acc: 0.8545
Epoch 65/500


209/209 [==============================] - 26s 123ms/step - loss: 0.3851 - acc: 0.8543
Epoch 136/500
209/209 [==============================] - 26s 126ms/step - loss: 0.3848 - acc: 0.8544
Epoch 137/500
209/209 [==============================] - 25s 121ms/step - loss: 0.3853 - acc: 0.8540
Epoch 138/500
209/209 [==============================] - 26s 123ms/step - loss: 0.3850 - acc: 0.8541
Epoch 139/500
209/209 [==============================] - 26s 123ms/step - loss: 0.3840 - acc: 0.8545
Epoch 140/500
209/209 [==============================] - 27s 129ms/step - loss: 0.3821 - acc: 0.8554
Epoch 141/500
209/209 [==============================] - 27s 127ms/step - loss: 0.3825 - acc: 0.8552
Epoch 142/500
209/209 [==============================] - 25s 117ms/step - loss: 0.3832 - acc: 0.8544
Epoch 143/500
209/209 [==============================] - 27s 127ms/step - loss: 0.3835 - acc: 0.8541
Epoch 144/500
209/209 [==============================] - 25s 120ms/step - loss: 0.3820 - acc: 0.8544
Epoc

209/209 [==============================] - 25s 122ms/step - loss: 0.3655 - acc: 0.8545
Epoch 217/500
209/209 [==============================] - 26s 126ms/step - loss: 0.3643 - acc: 0.8551
Epoch 218/500
209/209 [==============================] - 26s 123ms/step - loss: 0.3624 - acc: 0.8561
Epoch 219/500
209/209 [==============================] - 25s 118ms/step - loss: 0.3612 - acc: 0.8571
Epoch 220/500
209/209 [==============================] - 25s 121ms/step - loss: 0.3626 - acc: 0.85643s 
Epoch 221/500
209/209 [==============================] - 24s 117ms/step - loss: 0.3644 - acc: 0.8545
Epoch 222/500
209/209 [==============================] - 25s 119ms/step - loss: 0.3642 - acc: 0.8546
Epoch 223/500
209/209 [==============================] - 25s 120ms/step - loss: 0.3640 - acc: 0.8547
Epoch 224/500
209/209 [==============================] - 25s 121ms/step - loss: 0.3636 - acc: 0.8547
Epoch 225/500
209/209 [==============================] - 26s 125ms/step - loss: 0.3631 - acc: 0.8548
E

209/209 [==============================] - 25s 120ms/step - loss: 0.3501 - acc: 0.8556
Epoch 297/500
209/209 [==============================] - 26s 124ms/step - loss: 0.3513 - acc: 0.8551
Epoch 298/500
209/209 [==============================] - 24s 114ms/step - loss: 0.3499 - acc: 0.8557
Epoch 299/500
209/209 [==============================] - 24s 114ms/step - loss: 0.3487 - acc: 0.8563
Epoch 300/500
209/209 [==============================] - 26s 125ms/step - loss: 0.3477 - acc: 0.8573
Epoch 301/500
209/209 [==============================] - 26s 125ms/step - loss: 0.3478 - acc: 0.8572
Epoch 302/500
209/209 [==============================] - 26s 123ms/step - loss: 0.3498 - acc: 0.8553
Epoch 303/500
209/209 [==============================] - 23s 112ms/step - loss: 0.3496 - acc: 0.8554
Epoch 304/500
209/209 [==============================] - 25s 119ms/step - loss: 0.3478 - acc: 0.8565
Epoch 305/500
209/209 [==============================] - 25s 118ms/step - loss: 0.3478 - acc: 0.8565
Epoc

209/209 [==============================] - 26s 127ms/step - loss: 0.3405 - acc: 0.8563
Epoch 377/500
209/209 [==============================] - 24s 115ms/step - loss: 0.3394 - acc: 0.8567
Epoch 378/500
209/209 [==============================] - 26s 125ms/step - loss: 0.3385 - acc: 0.8572
Epoch 379/500
209/209 [==============================] - 25s 117ms/step - loss: 0.3376 - acc: 0.8577
Epoch 380/500
209/209 [==============================] - 25s 120ms/step - loss: 0.3370 - acc: 0.8583
Epoch 381/500
209/209 [==============================] - 25s 121ms/step - loss: 0.3370 - acc: 0.8588
Epoch 382/500
209/209 [==============================] - 26s 123ms/step - loss: 0.3387 - acc: 0.8565
Epoch 383/500
209/209 [==============================] - 28s 134ms/step - loss: 0.3380 - acc: 0.8567
Epoch 384/500
209/209 [==============================] - 26s 125ms/step - loss: 0.3388 - acc: 0.8563
Epoch 385/500
209/209 [==============================] - 25s 120ms/step - loss: 0.3381 - acc: 0.8566
Epoc

209/209 [==============================] - 25s 118ms/step - loss: 0.3298 - acc: 0.8574
Epoch 458/500
209/209 [==============================] - 25s 119ms/step - loss: 0.3294 - acc: 0.8575
Epoch 459/500
209/209 [==============================] - 25s 119ms/step - loss: 0.3290 - acc: 0.8576
Epoch 460/500
209/209 [==============================] - 25s 119ms/step - loss: 0.3291 - acc: 0.8577
Epoch 461/500
209/209 [==============================] - 26s 124ms/step - loss: 0.3280 - acc: 0.8580
Epoch 462/500
209/209 [==============================] - 26s 123ms/step - loss: 0.3271 - acc: 0.8584
Epoch 463/500
209/209 [==============================] - 25s 121ms/step - loss: 0.3265 - acc: 0.8587
Epoch 464/500
209/209 [==============================] - 26s 124ms/step - loss: 0.3255 - acc: 0.8592
Epoch 465/500
209/209 [==============================] - 26s 126ms/step - loss: 0.3251 - acc: 0.8597
Epoch 466/500
209/209 [==============================] - 24s 115ms/step - loss: 0.3258 - acc: 0.8599
Epoc

level 2
[[19475 19382]
 [ 2524  2619]] 0.5 0.51
[[19376 19388]
 [ 2634  2602]] 0.5 0.5
[[19566 19631]
 [ 2422  2381]] 0.5 0.5
[[19435 19545]
 [ 2516  2504]] 0.5 0.5
level 3
[[19329 19105]
 [ 2750  2816]] 0.5 0.5
[[19050 19452]
 [ 2749  2749]] 0.5 0.5
[[19315 19537]
 [ 2559  2589]] 0.5 0.5
[[19000 19603]
 [ 2705  2692]] 0.49 0.5
level 4
[[18891 19023]
 [ 3067  3019]] 0.5 0.5
[[18770 19128]
 [ 3099  3003]] 0.49 0.49
[[19210 19309]
 [ 2768  2713]] 0.5 0.5
[[18952 19030]
 [ 2958  3060]] 0.5 0.5
level 5
[[18512 18597]
 [ 3454  3437]] 0.5 0.5
[[18694 18576]
 [ 3435  3295]] 0.5 0.5
[[18894 18841]
 [ 3155  3110]] 0.5 0.5
[[18697 18477]
 [ 3461  3365]] 0.5 0.5
level 6
[[18198 18037]
 [ 3912  3853]] 0.5 0.5
[[17928 18475]
 [ 3755  3842]] 0.49 0.5
[[18345 18466]
 [ 3636  3553]] 0.5 0.5
[[18130 18146]
 [ 3887  3837]] 0.5 0.5
level 7
[[17760 17847]
 [ 4177  4216]] 0.5 0.5
[[17680 17844]
 [ 4293  4183]] 0.5 0.5
[[18200 17791]
 [ 3955  4054]] 0.51 0.51
[[17809 17737]
 [ 4180  4274]] 0.5 0.5
level 8
[